# Проектна задача по предметот „Мобилна роботика“
## Теодора Петровска, 98/2021
#### Во оваа тетратка е решен 3Д проблем за слетување на ракета инспириран од LunarLander од OpenAI Gym

Тестирање на работата на моторите

In [ ]:
from engine_tester import run_test
run_test(main=False, right=False, left=True, forward=False, back=False, steps=1000) # left thruster only

Импортирање на потребните библиотеки

In [1]:
import random
import statistics
from collections import deque
import threading
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
import mujoco
from mujoco import MjModel, MjData
import mujoco_viewer

Дефинирање на хиперпараметрите
- ALPHA: стапка на учење, колку брзо учи моделот
- GAMMA: фактор на дисконтирање, ги балансира краткотрајните и долготрајните награди. 0.995 значи дека битни ни се идните награди
- EPSILON*: го контролира истражувањето на агентот. Почни случајно да истражуваш, стани алчен со тек на време
- BATCH SIZE: од колку примероци да учи одеднаш
- BUFFER CAPACITY: максимална големина на меморијата
- TARGET UPDATE FREQ: фреквенција на синхронизирање на посакуваниот модел. На секои 10 епизоди ги копираме (синхронизираме) тежините од policy_net (онаа што учи) до target_net (онаа што се користи за пресметување на целната Q-вредност).
- TAU се користи за soft updates, бавно мешање на target и policy net

In [2]:
# Hyperparameters
ALPHA = 1e-4
GAMMA = 0.995
EPSILON_START = 1.0
EPSILON_END = 0.01
EPSILON_DECAY = 0.995
BATCH_SIZE = 256
BUFFER_CAPACITY = 200_000
TARGET_UPDATE_FREQ = 10
TAU = 0.005

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Користиме Deep Q-Network (длабока Q мрежа) - вид на невронска мрежа што се користи за апроксимација на Q функцијата, којашто ни кажува колку е добро да се преземе акција а во состојба ѕ.  
3D околината се состои од 13 влезови што ја опишуваат моменталната состојба на ракетата:
- x позиција (лево/десно)
- у позиција (право/позади)
- z позиција (нагоре/надолу)
- vх линеарна брзина по x
- vy линеарна брзина по y
- vz линеарна брзина по z
- wх аголна брзина по x - агол на валање (roll)
- wy аголна брзина по y - агол на скршнување (pitch)
- wz аголна брзина по z - агол на извишување (yaw)
- кватернион w (ротациски дел)
- кватернион х
- кватернион y
- кватернион z
  
Има 6 дискретни излези:
- 0: не прави ништо
- 1: активирај го главниот мотор
- 2: активирај го десниот мотор
- 3: активирај го левиот мотор
- 4: активирај го предниот мотор 
- 5: активирај го задниот мотор
  
Невронската мрежа се состои од следниве слоеви:
- Влезен слој 13 -> 518 неврони: ги трансформира 13-те влезни вредности во 512 димензионална репрезентација
- Скриен слој 512 -> 256: го намалуваме бројот бидејќи 512 е добро на почеток поради тоа што гледа многу различни шеми, а после 256 ја принудува мрежата да ги компресира тие податоци и да научи попрефинета репрезентација.
- Излезен слој 256 -> 6: дава една Q вредност по можна акција
- Нормализација: го нормализира излезот од претходниот слој, корисно е за високодимензионални и динамични состојби. Помага во стабилизирање на учењето.
- Активациска функција ReLU: се користи за да се воведе нелинеарност. Ова ѝ овозможува на мрежата да учи сложени шеми

Со методот „forward“ PyTorch знае како да ја води мрежата напред: зема состојба х и дава Q вредности за секоја акција. На пример ако состојбите на ракетата се:  

x = [0.1, 0.5, 12.3, 0.99, 0.01, 0.0, 0.01, -0.2, 0.0, -0.1, 0.0, 0.0, 0.01]

Тогаш  

model = DQN(13, 6)  
output = model.forward(x)  

Ќе даде нешто вакво  

output = [2.1, -0.5, 0.7, 1.4, -1.2, 0.3]

Што значи:  
- Q вредност не прави ништо: 2.1
- Q вредност активирај го главниот мотор: -0.5
- Q вредност активирај го десниот мотор: 0.7
- Q вредност активирај го левиот мотор: 1.4
- Q вредност активирај го предниот мотор : -1.2
- Q вредност активирај го задниот мотор: 0.3

Агентот ќе ја избере акцијата со најголема Q вредност што во случајов е да не прави ништо.

In [3]:
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.net(x)

Во Reinforcement Learning агентите учат од своите искуства (состојба, акција, награда, следна состојба и сл.). Но, ако учиме од нив по редоследот по којшто се случуваат, постои проблем каде што последователните искуства се многу корелирани, па, наместо веднаш да учиме, ние ги складираме искуствата во бафер и потоа случајно земаме примероци за да ја тренираме мрежата. Ова го надминува проблемот на корелација и го прави учењето поефикасно. 

Во иницијализацијата користиме deque (double-ended-queue) за да ги зачуваме искуствата. maxlen=capacity осигурува дека баферот не расте бесконечно, старите искуствата автоматски се отфрлаат кога ќе се наполни баферот.

Push методот додава едно искуство во баферот. Секое искуство е торка од 6 елементи: (self, state, action, reward, next_state, done). Секој пат кога агентот прави чекор во епизодата, тука го собираме неговото искуство.

Во sample случајно избираме искуства од batch_size од баферот. zip(*batch) ги дели торките во посебни листи: една листа од состојби, една од акции и сл. Ова ни овозможува да учиме од повеќе минати искуства одеднаш. Потоа, ги форматираме податоците во numpy низи за да можеме да ги конвертираме во тензори.

In [4]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        states, actions, rewards, next_states, dones = zip(*random.sample(self.buffer, batch_size))
        return (np.array(states),
                np.array(actions),
                np.array(rewards, dtype=np.float32),
                np.array(next_states),
                np.array(dones, dtype=np.uint8))

Креираме нова gym околина.

init методот ги гради ракетата, теренот и светот. Целиот 3Д свет е дефиниран користејќи MuJoCo XML:
- hfield: нерамен терен (128х128 heightmap)
- launch_pad: мала подигната платформа
- ракета дефинирана со:
  - главно тело во форма на цилиндар
  - врв на капсула
  - 3 ногарки на капсулата

Кога ќе ја вчитаме ракетата во MuJoCo имаме пристап до:
- self.data.qpos: позиција и ориентација на ракетата
- self.data.qvel: линеарна и аголна брзина
- self.data.qfrc_applied: примена на сили

Во reinforcement learning reset() прави 3 работи:
- ја става околината во нова состојба
- прави некои случајни работи (за да поттикне генерализација)
- ја враќа почетната опсервација на агентот

Методот reset() ги ресетира сите податоци за симулација во MuJoCo како да се почнува од 0, но истовремено додава и случајни иницијални линеарна и аголна брзина. Без овие случајни брзини ракетата секогаш би стартувала совршено мирно и би научила да слетува само од идеална состојба. Со оваа случајност агентот учи да се опорави од случајни реални ситуации (како на пр. талкање настрана, превртување) што го прави агентот многу попаметен и погенерализиран.

Со помош на get_obs() го добиваме влезот во невронската мрежа, односно комплетната обсервација (набљудување) што го има направено лендерот. Кватернионите мора да ги нормализираме пред да ги користиме. 

Функцијата step() прави 5 големи работи:
1. Ја претвора акцијата на агентот во сила
2. Ја применува таа сила во MuJoCo
3. Ја унапредува симулацијата за еден чекор
4. Ја пресметува наградата
5. Проверува дали епизодата е завршена

Прво, ориентацијата на ракетата ја претвораме во 3х3 матрица на ротација така што можеме да примениме сила во вистинската насока, дури и ако е навалена ракетата. Односно, ако директно сакаме да примениме сила по z-оска на пример, доколку ракетата е навалена силата ќе влијае во погрешна насока бидејќи таа ќе ја гледа z-оската во однос на нејзиниот свет, не глобалниот. Затоа ни треба матрицата на ротација. Таа опишува како да се ротира вектор од еден координатен систем во друг.

compute_reward() пресметува колку добро се справува агентот врз основа на неговата позиција, брзина, ориентација и растојание од платформата. 
- tilt_penalty: ја казнува ракетата за преголемо навалување. Ако qx и qy се 0, таа е совршено исправена
- dist: колу е оддалечена ракетата од центарот на платформата
- vel_horizontal: колку брзо се движи од страна на страна
- vel_vertical: колку брзо се движи нагоре/надолу
- altitude_reward: колку е блиску до платформата
- position_reward: целта е ракетата да е поставена директно над платформата
- velocity_reward: движи се бавно и сигурно кога слетуваш
- orientation_reward: остани исправена

is_done се користи за да дознаеме дали обидот на ракетата да слета заврши.

render() функцијата визуелно ја прикажува симулацијата. Отвора нов прозорец и ја прикажува ракетата како лета наоколу во 3Д. Кога ќе заврши визуелизацијата, се затвора со close().

In [5]:
class LunarLander3D(gym.Env):
    metadata = {"render_modes": ["human"], "render_fps": 60}

    def __init__(self, render_mode="human"):
        super().__init__()
        self.render_mode = render_mode

        xml = """
        <mujoco>
          <asset>
            <hfield name="terrain" nrow="128" ncol="128" size="40 40 2 0.1"/>
            <material name="moon_soil" rgba="0.3 0.3 0.3 1"/>
            <material name="launch_pad" rgba="0.8 0.1 0.1 1"/>
            <material name="rocket_body" rgba="0.9 0.9 0.9 1"/>
            <material name="legs" rgba="0.2 0.2 0.2 1"/>
          </asset>
          <worldbody>
            <light directional="true" diffuse="0.9 0.9 0.9" pos="0 0 10"/>
            <geom type="hfield" hfield="terrain" material="moon_soil" pos="0 0 0"/>
            <body pos="0 0 0.5">
              <geom type="cylinder" size="2 0.3" material="launch_pad"/>
            </body>
            <body name="lander" pos="0 0 15">
              <joint name="root" type="free"/>
              <geom type="cylinder" size="0.4 6" pos="0 0 0" material="rocket_body"/>
              <geom type="capsule" fromto="0 0 6 0 0 8" size="0.3" material="rocket_body"/>
              <geom type="capsule" fromto="0.6 0 -5.5 1.0 0 -8.0" size="0.1" material="legs"/>
              <geom type="capsule" fromto="-0.3 0.519 -5.5 -0.6 1.038 -8.0" size="0.1" material="legs"/>
              <geom type="capsule" fromto="-0.3 -0.519 -5.5 -0.6 -1.038 -8.0" size="0.1" material="legs"/>
              <geom type="sphere" size="0.15" pos="0.6 0 -5.5" material="legs"/>
              <geom type="sphere" size="0.15" pos="-0.3 0.519 -5.5" material="legs"/>
              <geom type="sphere" size="0.15" pos="-0.3 -0.519 -5.5" material="legs"/>
            </body>
          </worldbody>
        </mujoco>
        """

        self.model = mujoco.MjModel.from_xml_string(xml)
        self._generate_lunar_terrain()
        self.data = MjData(self.model)
        self.viewer = None

        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(-np.inf, np.inf, (13,), dtype=np.float32)
        self.max_steps = 1000
        self.step_count = 0

    def _generate_lunar_terrain(self):
        heightmap = np.zeros((128, 128), dtype=np.float32)
        for _ in range(5):
            x, y = np.random.randint(20, 108), np.random.randint(20, 108)
            size = np.random.uniform(5, 15)
            depth = np.random.uniform(0.5, 2.0)
            xx, yy = np.mgrid[:128, :128]
            dist = np.sqrt((xx - x) ** 2 + (yy - y) ** 2)
            heightmap += depth * np.exp(-dist ** 2 / (2 * (size ** 2)))
        heightmap += np.random.rand(128, 128) * 0.3
        heightmap[60:68, 60:68] = heightmap.min()
        heightmap = (heightmap - heightmap.min()) / (heightmap.max() - heightmap.min())
        heightmap = heightmap * 1.5 + 0.1
        self.model.hfield_data[:] = heightmap.ravel()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        self.step_count = 0
    
        # Linear velocities
        self.data.qvel[0] = self.np_random.uniform(-3.0, 3.0)  # X velocity (sideways)
        self.data.qvel[1] = self.np_random.uniform(-3.0, 3.0)  # Y velocity (forward/back)
        self.data.qvel[2] = self.np_random.uniform(-1.0, 1.0)  # Z velocity (up/down)
        
        ## Angular velocities
        #self.data.qvel[3] = self.np_random.uniform(-0.5, 0.5)  # X-axis rotation (roll)
        #self.data.qvel[4] = self.np_random.uniform(-0.5, 0.5)  # Y-axis rotation (pitch)
        #self.data.qvel[5] = self.np_random.uniform(-0.5, 0.5)  # Z-axis rotation (yaw)
        # No angular velocity — don't spin
        self.data.qvel[3:6] = 0

        
        return self._get_obs(), {}

    def _get_obs(self):
        # Position (x, y, z) and quaternion
        position = self.data.qpos[0:3]
        quat = self.data.qpos[3:7]
    
        # Normalize quaternion
        quat /= np.linalg.norm(quat)
    
        # Velocity (linear and angular)
        linear_vel = self.data.qvel[0:3]
        angular_vel = self.data.qvel[3:6]
    
        return np.concatenate([
            position,
            quat,
            linear_vel,
            angular_vel
        ]).astype(np.float32)

    def step(self, action):
        # Get body rotation matrix
        body_quat = self.data.body("lander").xquat  # [w, x, y, z] format
        body_rot = np.zeros(9)
        mujoco.mju_quat2Mat(body_rot, body_quat)
        rotation_matrix = body_rot.reshape(3, 3)
        
        # Apply thrust in body frame
        if action == 1:          # main
            body_thrust = np.array([  0.0,   0.0, -180.0])
        elif action == 2:        # right
            body_thrust = np.array([ 60.0,   0.0,    0.0])
        elif action == 3:        # left
            body_thrust = np.array([-60.0,   0.0,    0.0])
        elif action == 4:        # forward (+y)
            body_thrust = np.array([  0.0,  60.0,    0.0])
        elif action == 5:        # back (−y)
            body_thrust = np.array([  0.0, -60.0,    0.0])
        else:                    # action == 0
            body_thrust = np.zeros(3)

        global_thrust = rotation_matrix @ body_thrust

    
        # Apply the transformed force
        mujoco.mj_applyFT(
            self.model, self.data, 
            global_thrust, 
            np.zeros(3),
            np.zeros(3),
            mujoco.mj_name2id(self.model, mujoco.mjtObj.mjOBJ_BODY, b"lander"), 
            self.data.qfrc_applied
        )
        
        # Step the simulation and return values
        mujoco.mj_step(self.model, self.data)
        self.step_count += 1
        
        obs = self._get_obs()
        reward = self._compute_reward(obs)
        done = self._is_done(obs)
        
        return obs, reward, done, False, {}

    def _compute_reward(self, obs):
        x, y, z = obs[0:3]
        body_z = z
        feet_z = body_z - 8.0  # Adjust for body-foot offset

        # Quaternion components (already normalized)
        qw, qx, qy, qz = obs[3:7]
        lin_vel = obs[7:10]
        ang_vel = obs[10:13]
        
        # Use 2*(qx^2 + qy^2) for tilt (0=upright, 1=horizontal)
        tilt_penalty = 2 * (qx**2 + qy**2)
        
        dist = np.sqrt(x**2 + y**2)
        vel_horizontal = np.linalg.norm(lin_vel[:2])
        vel_vertical = lin_vel[2]

        # Base rewards
        altitude_reward = np.exp(-0.5 * abs(feet_z - 0.5))  # Peak at pad height
        position_reward = np.exp(-2 * dist)  # around pad
        velocity_reward = np.exp(-4 * vel_horizontal) - 0.5 * abs(vel_vertical)
        orientation_reward = 1 - tilt_penalty
    
        reward = (
            2.0 * altitude_reward +
            1.5 * position_reward +
            1.0 * velocity_reward +
            0.8 * orientation_reward
        )
        
        # Success bonus (upright, low speed, on pad)
        if (feet_z < 0.3 and dist < 0.5 and 
            vel_horizontal < 0.2 and abs(vel_vertical) < 0.3 and
            tilt_penalty < 0.1):
            reward += 200
        # Off-pad but landed gently = penalty
        elif feet_z < 0.3 and dist >= 0.5:
            reward -= 50

        
        # Crash penalty
        if feet_z < 0:
            reward -= 100
        
        return reward
        
    def _is_done(self, obs):
        feet_z = obs[2] - 8.0
        return feet_z < 0 or self.step_count >= self.max_steps
    
    def render(self):
        if self.viewer is None:
            self.viewer = mujoco_viewer.MujocoViewer(self.model, self.data)
            self.viewer.cam.distance = 25.0
            self.viewer.cam.azimuth = 35
            self.viewer.cam.elevation = -25
        self.viewer.render()

    def close(self):
        if self.viewer:
            self.viewer.close()

Класата Normalizer ги следи тековната средна вредност и стандардната девијација на состојбите и ги нормализира. Зошто да се нормализира? Невронските мрежи учат подобро кога влезните вредности се центрирани околу 0 и се во разумен опсег (не премногу големи, не премногу мали). Ако на пр. z=15 и qx=0.02 мрежата ќе се бори да ги балансира скаливе, освен ако не се нормализира.

update ги ажурира овие податоци базирано на нова обсервација x.

normalize ја центрира вредноста со тоа што ја одзема средната вредност и ја скалира вредноста со тоа што ја дели со std. 1e-8 се додава за да се спречи делење со 0. Резултат: влезот има средна вредност приближна на 0 и std приближно на 1 што е подобро за тренирање.

In [6]:
class Normalizer:
    def __init__(self, size):
        self.n = 0                       # n - number of updates so far
        self.mean = np.zeros(size)       # mean - mean vector (starts at 0)
        self.var = np.ones(size)         # var - variance (starts at 1)
        self.std = np.sqrt(self.var)     # std - standard deviation (sqrt of variance)

    def update(self, x):
        self.n += 1
        new_mean = self.mean + (x - self.mean) / self.n
        new_var = self.var + (x - self.mean) * (x - new_mean)
        self.mean = new_mean
        self.var = new_var
        self.std = np.sqrt(self.var / self.n) if self.n > 1 else np.ones_like(x)

    def normalize(self, x):
        return (x - self.mean) / (self.std + 1e-8)  # Prevent division by zero

Класата Trainer поставува 3D околина, иницијализира мрежи, бафер и оптимизатор, тренира, исцртува награди, го визуелизира агентот...

init() ја иницијализира 3Д околината. Со render_mode=None викаме дека нема прозорец за прегледување (побрзо се тренира). Ги оптимизира тежините со помош на Адам.

policy() ја скалира 13 димензионалната состојба да има средна вредност 0 и стандардна девијација 1. Осигурува дека различни влезни карактеристики (како позиција, брзина) нема да доминираат во учењето. Во раните фази на тренирање, истражувањето е поттикнато бидејќи имаме веројатност епсилон за случајно дејство. Инаку, се избира најпознатата акција од Q мрежата. Нормализираната состојба се префрла во PyTorch тензор и се прави unsqueeze(0) за обликот да стане [1,13]. Тука немаме градиенти бидејќи не тренираме, само евалуираме. На крај, argmax(dim=1) ја избира акцијата со највисока предвидена Q одговорност. item() го претвора тензорот во обичен број.

update_model() прво проверува дали има доволно примероци за тренирање, ако баферот не е полн не тренираме. Земаме batch, нормализираме и конвертираме во PyTorch тензори. Потоа, ги предвидуваме сите Q вредности од моменталните состојби. Користиме .gather() за да ги земеме Q вредностите за преземените акции. Користиме Double DQN со тоа што земаме argmax од policy_net (кое дејство би го избрале), но добиваме вредност од target_net (постабилно е). Грешката се рачуна како MSE на предвидените Q вредности и целните Q вредности. Тука имаме и бекпропагација со тоа што ги чистиме старите градиенти, калкулираме нови, овие градиенти и кажуваат на мрежата како да ги промени своите тежини за да се намали загубата, ги кратиме градиенти за да не станат преголеми и ги ажурираме тежините.

Во train() е целата логика за тренирање на моделот. Се повикуваат претходно објаснетите функции и на секои 20 епизоди се визуелизира дотогап најдоброто слетување на ракетата.

In [7]:
class Trainer:
    def __init__(self):
        self.env = LunarLander3D(render_mode=None)  # Disable rendering during training
        self.policy_net = DQN(13, 6).to(device)
        self.target_net = DQN(13, 6).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=ALPHA)
        self.replay_buffer = ReplayBuffer(BUFFER_CAPACITY)
        self.epsilon = EPSILON_START
        self.best_reward = -float('inf')
        self.should_stop = False
        self.rewards = []
        self.setup_dashboard()
        self.normalizer = Normalizer(13)

    def policy(self, state):
        normalized_state = self.normalizer.normalize(state)
        # ε-greedy over Discrete(4)
        if random.random() < self.epsilon:
            return self.env.action_space.sample()   # random int in [0..3]
        # else pick argmax Q
        state_tensor = torch.FloatTensor(normalized_state).unsqueeze(0).to(device)
        with torch.no_grad():
            q_vals = self.policy_net(state_tensor)  # shape [1,4]
        return int(q_vals.argmax(dim=1).item())


    def setup_dashboard(self):
        self.fig = go.FigureWidget()
        self.fig.add_scatter(x=[], y=[], mode='markers+lines')
        self.fig.layout.title = '3D Lunar Lander Training'
        self.fig.layout.xaxis.title = 'Episode'
        self.fig.layout.yaxis.title = 'Reward'
        
        self.stop_button = widgets.Button(description='Stop Training', button_style='danger')
        self.stop_button.on_click(lambda b: setattr(self, 'should_stop', True))
        
        self.dashboard = widgets.VBox([
            self.stop_button,
            self.fig
        ])

    def update_model(self):
        # Don’t train until we have at least one full batch
        if len(self.replay_buffer.buffer) < BATCH_SIZE:
            return

        # Sample and normalize
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(BATCH_SIZE)

        # Normalize all states
        normalized_states = np.array([self.normalizer.normalize(s) for s in states])
        normalized_next_states = np.array([self.normalizer.normalize(s) for s in next_states])

        states      = torch.FloatTensor(normalized_states).to(device)
        actions     = torch.LongTensor(actions).view(-1,1).to(device)    
        rewards     = torch.FloatTensor(rewards).unsqueeze(1).to(device) 
        next_states = torch.FloatTensor(normalized_next_states).to(device)
        dones       = torch.FloatTensor(dones).unsqueeze(1).to(device)   

        q_vals    = self.policy_net(states)                       
        current_q = q_vals.gather(1, actions)                     

        with torch.no_grad():
            # Double DQN: Use policy net for action, target net for value
            next_actions = self.policy_net(next_states).argmax(dim=1, keepdim=True)
            max_next_q = self.target_net(next_states).gather(1, next_actions)
            target_q = rewards + GAMMA * max_next_q * (1 - dones)

        loss = nn.MSELoss()(current_q, target_q)
        self.optimizer.zero_grad()  # clear old gradients
        loss.backward()  # backpropagation
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), max_norm=1.0)  # clip gradients so they don't get too large
        self.optimizer.step()   # update weights

    def train(self, num_episodes=1000):
        display(self.dashboard)
        
        # Initialize normalizer with first state
        state, _ = self.env.reset()
        self.normalizer.update(state)  # Initial state
        
        for episode in range(num_episodes):
            if self.should_stop:
                print("Training stopped by user")
                break
                
            state, _ = self.env.reset()
            total_reward = 0
            done = False
            
            while not done and not self.should_stop:
                # Normalize state before passing to policy
                normalized_state = self.normalizer.normalize(state)
                action = self.policy(normalized_state)  # Policy now uses normalized state
                
                next_state, reward, done, _, _ = self.env.step(action)
                
                # Update normalizer with new state
                self.normalizer.update(next_state)
                
                self.replay_buffer.push(state, action, reward, next_state, done)
                self.update_model()
                total_reward += reward
                state = next_state

            self.epsilon = max(EPSILON_END, self.epsilon * EPSILON_DECAY)
            
            for target_param, policy_param in zip(self.target_net.parameters(), self.policy_net.parameters()):
                target_param.data.copy_(TAU * policy_param.data + (1 - TAU) * target_param.data)

            if total_reward > self.best_reward:
                self.best_reward = total_reward
                torch.save(self.policy_net.state_dict(), "best_3d_lander.pth")
                print(f"New Best Model saved at Episode {episode} with Reward {total_reward:.2f}")

            # Update dashboard
            with self.fig.batch_update():
                self.fig.data[0].x = tuple(list(self.fig.data[0].x) + [episode])
                self.fig.data[0].y = tuple(list(self.fig.data[0].y) + [total_reward])

            # Visualize in main thread
            if episode % 20 == 0:
                self.visualize_agent()
                
        self.env.close()

    def visualize_agent(self):
        preview_env = LunarLander3D(render_mode="human")
        # load best weights once
        self.policy_net.load_state_dict(torch.load("best_3d_lander.pth"))
        self.policy_net.eval()

        state, _ = preview_env.reset()
        done = False

        while not done:
            # ε=0 greedy
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            with torch.no_grad():
                action = int(self.policy_net(state_tensor).argmax().item())

            # step + render
            state, reward, done, _, _ = preview_env.step(action)
            preview_env.render()

            # cap FPS
            import time
            time.sleep(1/60)

        preview_env.close()

if __name__ == "__main__":
    trainer = Trainer()
    trainer.train(num_episodes=1000)

New Best Model saved at Episode 0 with Reward 123.96
New Best Model saved at Episode 1 with Reward 264.33
New Best Model saved at Episode 3 with Reward 844.75
New Best Model saved at Episode 120 with Reward 960.59
New Best Model saved at Episode 201 with Reward 1276.46
New Best Model saved at Episode 760 with Reward 1676.48
New Best Model saved at Episode 841 with Reward 1814.18
